In [0]:
!pip install -U keras

In [0]:
!pip install git+https://github.com/JihongJu/keras-resnet3d.git

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import keras
from keras.layers import Dense,Conv2D,LSTM,Bidirectional,TimeDistributed,Input,RepeatVector
from keras.models import Model
from keras.optimizers import RMSprop

In [0]:
#Importing the Data 
X_train = np.load('/content/drive/My Drive/Sign Language Recognition/X_train.npy')
y_train = np.load('/content/drive/My Drive/Sign Language Recognition/y_train.npy')
print(X_train.shape)
print(y_train.shape)

In [0]:
input_shape = (38,24,8,224,224,3)
out_timesteps = 28
out_vocab = 1232

In [0]:
# Defining the resnet3d model
from resnet3d import Resnet3DBuilder
resnet3dmodel = Resnet3DBuilder.build_resnet_18((input_shape[2], input_shape[3], input_shape[4], input_shape[5]), 0)
# layer_names = [layer.name for layer in model.layers]
last_layer = resnet3dmodel.layers[-2]
base_model = Model(inputs = resnet3dmodel.input, outputs=last_layer.output)

In [0]:
# Defining the main model

#Input layer
input_tensor = Input(shape=(input_shape[1],input_shape[2],input_shape[3],input_shape[4],input_shape[5]))

#Feature Extractor
resnet = TimeDistributed(base_model)(input_tensor)

#Encoder
bidirectional_lstm_1 = Bidirectional(LSTM(1024))(resnet)

#Decoder
repeatVector = RepeatVector(out_timesteps)(bidirectional_lstm_1)
lstm_1 = LSTM(1024, return_sequences=True)(repeatVector)

#Output Dense layer
output_tensor = Dense(out_vocab, activation='softmax')(lstm_1)
print(output_tensor)

model = Model(inputs=input_tensor,outputs=output_tensor)

In [0]:
#Model compilation
rms = RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [0]:
# filename = 'model.h1.6_april_20'
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(X_train, y_train.reshape(y_train.shape[0], y_train.shape[1], 1),
                    epochs=30, batch_size=2, validation_split = 0.2, 
                    verbose=1,shuffle=True)